In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.utils import plot_model
import pydot
import graphviz
import joblib

In [2]:
# Load the dataset
data = pd.read_csv('BLCA_GSE130001_AllDiffGenes_table.tsv',sep='\t')

In [3]:
data

,Cluster,Celltype (malignancy),Celltype (major-lineage),Celltype (minor-lineage),Gene,log2FC,Percentage (%),Adjusted p-value
0,0,Stromal cells,Epithelial,Epithelial,IGFBP3,1.07,99.6,4.820000e-198
1,0,Stromal cells,Epithelial,Epithelial,FN1,1.20,95.9,4.320000e-166
2,0,Stromal cells,Epithelial,Epithelial,S100A6,0.66,100.0,2.310000e-122
3,0,Stromal cells,Epithelial,Epithelial,CAV1,0.81,92.9,8.460000e-101
4,0,Stromal cells,Epithelial,Epithelial,PERP,0.53,96.9,2.910000e-90
...,...,...,...,...,...,...,...,...
10052,9,Stromal cells,Epithelial,Epithelial,RAC1,-0.32,79.4,6.510000e-06
10053,9,Stromal cells,Epithelial,Epithelial,KBTBD6,-0.44,1.0,7.030000e-06
10054,9,Stromal cells,Epithelial,Epithelial,SRSF5,-0.33,57.7,8.270000e-06
10055,9,Stromal cells,Epithelial,Epithelial,LDHA,-0.43,52.6,8.740000e-06


In [4]:
# Select features and target
features = data[['Cluster', 'Celltype (malignancy)', 'Celltype (major-lineage)', 'Celltype (minor-lineage)', 'Gene', 'Percentage (%)', 'Adjusted p-value']]
target = data['log2FC']

In [5]:
target

0        1.07
1        1.20
2        0.66
3        0.81
4        0.53
         ... 
10052   -0.32
10053   -0.44
10054   -0.33
10055   -0.43
10056   -0.49
Name: log2FC, Length: 10057, dtype: float64

In [6]:
# Handle categorical features
categorical_features = ['Cluster', 'Celltype (malignancy)', 'Celltype (major-lineage)', 'Celltype (minor-lineage)', 'Gene']
numeric_features = ['Percentage (%)', 'Adjusted p-value']

In [7]:
# Define column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [8]:
# Fit the preprocessor on the entire dataset
preprocessor.fit(features)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['Percentage (%)', 'Adjusted p-value']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['Cluster', 'Celltype (malignancy)',
                                  'Celltype (major-lineage)',
                                  'Celltype (minor-lineage)', 'Gene'])])

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [10]:
# Preprocess the data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [11]:
# Save the preprocessor
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']

In [12]:
# Define models and hyperparameters
models = {
    'Ridge': Ridge(),
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'SVR': SVR()
}

In [13]:
# Train and tune models
best_models = {}
best_params = {}
best_scores = {}

In [14]:
# Define parameter grids for each model
param_grids = {
    'Ridge': {
        'alpha': [0.1, 1, 10, 100]  # Regularization strength
    },
    'RandomForest': {
        'n_estimators': [50, 100, 200],  # Number of trees
        'max_depth': [None, 10, 20, 30],  # Maximum depth of trees
        'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
        'min_samples_leaf': [1, 2, 4]  # Minimum samples required to be at a leaf node
    },
    'GradientBoosting': {
        'n_estimators': [50, 100, 200],  # Number of boosting stages to be run
        'learning_rate': [0.01, 0.1, 0.2],  # Step size for each iteration
        'max_depth': [3, 5, 7]  # Maximum depth of the individual trees
    },
    'SVR': {
        'C': [0.1, 1, 10, 100],  # Regularization parameter
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
        'gamma': ['scale', 'auto']  # Kernel coefficient
    }
}

In [15]:
for model_name, model in models.items():
    print(f"Training {model_name}...")
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    grid_search = GridSearchCV(pipeline, {'model__' + k: v for k, v in param_grids[model_name].items()}, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    best_models[model_name] = grid_search.best_estimator_
    best_params[model_name] = grid_search.best_params_
    best_scores[model_name] = -grid_search.best_score_
    
    print(f"Best params for {model_name}: {best_params[model_name]}")
    print(f"Best score for {model_name}: {best_scores[model_name]}")


Training Ridge...
Best params for Ridge: {'model__alpha': 0.1}
Best score for Ridge: 0.1971994921164279
Training RandomForest...
Best params for RandomForest: {'model__max_depth': 30, 'model__min_samples_leaf': 1, 'model__min_samples_split': 10, 'model__n_estimators': 200}
Best score for RandomForest: 0.31488532059035956
Training GradientBoosting...
Best params for GradientBoosting: {'model__learning_rate': 0.2, 'model__max_depth': 7, 'model__n_estimators': 200}
Best score for GradientBoosting: 0.2753331050472815
Training SVR...
Best params for SVR: {'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'linear'}
Best score for SVR: 0.21134517377743506


In [16]:
# Define a simple neural network model
def create_nn_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train_transformed.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


In [17]:
# Train Neural Network
from scikeras.wrappers import KerasRegressor
nn_model = KerasRegressor(build_fn=create_nn_model, epochs=100, batch_size=10, verbose=1)

In [18]:
# Save the best model (example: Ridge)
nn_model.fit(X_train_transformed, y_train)

/Users/pritam/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/pritam/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.3948
Epoch 2/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1912
Epoch 3/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0838
Epoch 4/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0462
Epoch 5/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0367
Epoch 6/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0267
Epoch 7/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0211
Epoch 8/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0188
Epoch 9/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0164
Epoch 10/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0144
Epoch 11/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0127
Epoch 12/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0114
Epoch 13/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0100
Epoch 14/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0102
Epoch 15/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2

805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0014
Epoch 81/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0013
Epoch 82/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0015
Epoch 83/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0014
Epoch 84/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0015
Epoch 85/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0013
Epoch 86/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0012
Epoch 87/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0014
Epoch 88/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0013
Epoch 89/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0012
Epoch 90/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0013
Epoch 91/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0012
Epoch 92/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0011
Epoch 93/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0011
Epoch 94/100
805/805 ━━━━━━━━━━━━━━━━━━━━ 2s 2m

KerasRegressor(
	model=None
	build_fn=<function create_nn_model at 0x171462480>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=10
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
)

In [19]:
joblib.dump(best_models['Ridge'], 'best_ridge_model_pipeline.pkl')

['best_ridge_model_pipeline.pkl']

In [20]:
best_ridge_model_pipeline = joblib.load('best_ridge_model_pipeline.pkl')

In [33]:
y_pred = best_ridge_model_pipeline.predict(X_test)


In [34]:
y_pred


array([ 0.34534794, -0.71726436, -1.25891832, ...,  0.59663259,
        0.50406476,  0.01581171])

In [35]:
# Create a DataFrame to compare predictions with actual values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Display the comparison
print(comparison_df.head())


      Actual  Predicted
8257    0.97   0.345348
5247   -0.29  -0.717264
1149   -1.39  -1.258918
8621    0.97   0.648093
107     0.38   0.338423


In [41]:
from sklearn.metrics import mean_squared_error, r2_score

# Load the saved pipeline
loaded_pipeline = joblib.load('best_ridge_model_pipeline.pkl')

# Make predictions
y_pred = loaded_pipeline.predict(X_test)

# Evaluate the predictions
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')

Mean Squared Error: 0.194412454624329
R² Score: 0.6309556602902455
